# Local Mode training and inference in SageMaker Studio
This Jupyter notebook is based on AWS  sagemaker local-mode example [xgboost_script_mode_local_training_and_serving](https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/599b86851ffb4c081b8f6b5458c98e9bf7479260/xgboost_script_mode_local_training_and_serving/xgboost_script_mode_local_training_and_serving.py)
The notebook has steps added to setup `local mode` on SageMaker Studio. This notebook is tested on `Python3 (Data Science)` kernel

## Intial setup
### Run `setup.sh` to execute initial setup
`setup.sh` will do the following:
- Create `~/.sagemaker_studio_docker_cli` directory
- Setup softlink for `sdocker` to make it possible to run it from anywhere from command line
- Install `docker` and `docker-compose`
- Create `~/temp` directory used in `local mode`
- Create `config.yaml` to change temporay directory to `~/temp`
- Install branch `remote_docker_host` from SageMaker Python SDK which introduces Remote Docker Host capability (see [PR 2864](https://github.com/aws/sagemaker-python-sdk/pull/2864))

In [ ]:
!cd .. ;./setup.sh

### Fix dependencies
`Python 3 (Data Science)` kernel comes with `pyyaml==6.0` and doesn't have `pgrep` or `procps`. `local mode` requires `pyyaml==5.4.1`, higher versions cause error. Also `pgrep` is required when deleting a local endpoint.

In [ ]:
!conda update --force -y conda
!conda install -y pyyaml==5.4.1
!apt-get install -y procps

#### **Important**
Restart kernel for the above installations to take effect

### Create Docker Host
`sdocker` help us provision an EC2 instance that we can use as a docker host to run docker commands remotely. `sdocker` does the following:
- Setup networking and security groups between the instance and SageMaker Studio Apps and EFS
- Provision EC2 instance
- Mount SageMaker Studio EFS on EC2 instance
- Create docker context to connect to docker host


In [ ]:
!sdocker create-host --instance-type c5.xlarge

#### Download dataset
Run cell below to download dataset

In [ ]:
!cd data/train; wget https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-local-mode/main/xgboost_script_mode_local_training_and_serving/data/train/abalone

#### Create XGBoost estimator

In [ ]:
from sagemaker import TrainingInput
from sagemaker.xgboost import XGBoost, XGBoostModel

DUMMY_IAM_ROLE = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

hyperparameters = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "reg:squarederror",
    "num_round": "50",
    "verbosity": "2",
}

xgb_script_mode_estimator = XGBoost(
    entry_point="./code/abalone.py",
    hyperparameters=hyperparameters,
    role=DUMMY_IAM_ROLE,
    instance_count=1,
    instance_type='local',
    framework_version="1.2-1"
)

#### Start local training

In [ ]:
train_input = TrainingInput("file://./data/train/abalone", content_type="text/libsvm")

xgb_script_mode_estimator.fit({"train": train_input, "validation": train_input})

#### Deploy local endpoint

In [ ]:
model_data = xgb_script_mode_estimator.model_data

xgb_inference_model = XGBoostModel(
    model_data=model_data,
    role=DUMMY_IAM_ROLE,
    entry_point="./code/inference.py",
    framework_version="1.2-1",
)

predictor = xgb_inference_model.deploy(
    initial_instance_count=1,
    instance_type="local"
)

#### Invoke local endpoint

In [ ]:
def do_inference_on_local_endpoint(predictor, libsvm_str):
    label, *features = libsvm_str.strip().split()
    predictions = predictor.predict(" ".join(["-99"] + features))  # use dummy label -99
    print()
    print("*********************************************************")
    print("Prediction: {}".format(predictions))
    print("*********************************************************")
    print()
    
a_young_abalone = "6 1:3 2:0.37 3:0.29 4:0.095 5:0.249 6:0.1045 7:0.058 8:0.067"
do_inference_on_local_endpoint(predictor, a_young_abalone)

an_old_abalone = "15 1:1 2:0.655 3:0.53 4:0.175 5:1.2635 6:0.486 7:0.2635 8:0.415"
do_inference_on_local_endpoint(predictor, an_old_abalone)

#### Clean up

In [ ]:
predictor.delete_endpoint(predictor.endpoint_name)

### Delete Current Docker Host
Once you finish with docker host, you can terminate the instance with the below command. Note this command will only work if a docker host was created successfully with `create-host` command. Otherwise, please make sure to terminate the instance manually to avoid incurring any extra costs.

In [ ]:
!sdocker terminate-current-host